In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

In [ ]:
train_data = pd.read_csv("data_cust_2-3_train.csv")
train_x = train_data.iloc[:, 2:]
train_y = get_one_hot(train_data.iloc[:, 1], 2)

In [ ]:
print(train_y[0:10])

In [ ]:
x = tf.placeholder(tf.float32, [None, len(train_x.columns)])
y = tf.placeholder(tf.float32, [None, 2]) 

In [ ]:
W = tf.Variable(tf.zeros([len(train_x.columns), 2]))
b = tf.Variable(tf.zeros([2]))

In [ ]:
h = tf.nn.softmax(tf.matmul(x, W) + b)
cross_entropy = -tf.reduce_sum(y * tf.log(h), reduction_indices=[1])
loss = tf.reduce_mean(cross_entropy)

In [ ]:
train = tf.train.GradientDescentOptimizer(0.000001).minimize(loss)

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
for i in range(1000001):
    _, loss_value = sess.run([train, loss], feed_dict={x:train_x, y:train_y})
    if i % 10000 == 0:
        print('Step %d, %.5f' % (i, loss_value))

In [ ]:
test_data = pd.read_csv("data_cust_2-3_test.csv")
test_x = test_data.iloc[:, 2:]
test_y = get_one_hot(test_data.iloc[:, 1], 2)

In [ ]:
correct_prediction = tf.equal(tf.argmax(h, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(sess.run(accuracy, feed_dict={x: test_x, y:test_y}))

In [ ]:
print(sess.run(h, feed_dict={x:test_x, y:test_y})[0:10])

In [ ]:
predict_y = sess.run(tf.argmax(h, 1), feed_dict={x:test_x, y:test_y})
print(predict_y[0:20])

In [ ]:
print(test_y.argmax(axis=1)[0:20])

In [ ]:
cross_tab = pd.crosstab(test_y.argmax(axis=1), predict_y, margins=True)
print(cross_tab)

In [ ]:
precision = cross_tab[1][1]/(cross_tab[1][0] + cross_tab[1][1])
print(precision)

In [ ]:
recall = cross_tab[1][1]/(cross_tab[0][1] + cross_tab[1][1])
print(recall)

In [ ]:
f_measure = (2*precision*recall) / (precision + recall)
print(f_measure)